In [1]:
from torchvision.models import mobilenet_v3_small
from torchvision.models.mobilenetv3 import MobileNet_V3_Small_Weights
import torch.nn as nn
import torch

In [2]:
import torch
print(torch.cuda.is_available())


True


In [3]:
mobilenet_small = mobilenet_v3_small(pretrained=True)

/gris/gris-f/homestud/jplapper/anaconda3/envs/signenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/gris/gris-f/homestud/jplapper/anaconda3/envs/signenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
mobilenet = mobilenet_small.to(device)


cuda


In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)


In [6]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Define the dataset path and set the batch size
dataset_path = '/gris/gris-f/homestud/jplapper/SignLanguageApp/SigNN Character Database'
batch_size = 64  # You can adjust this based on your system's memory


In [7]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [8]:
# Load the entire dataset
full_dataset = datasets.ImageFolder(dataset_path, transform=data_transforms['train'])

# Split the dataset into training and validation sets
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Now apply the appropriate transformations to each dataset split
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

dataloaders = {'train': train_loader, 'val': val_loader}


In [11]:
import torch.nn as nn
import torchvision.models as models

mobilenet = models.mobilenet_v3_small(pretrained=True)
for param in mobilenet.parameters():
    param.requires_grad = False

# Adjust the final layer according to the number of classes in your dataset
num_ftrs = mobilenet.classifier[3].in_features
NUM_CLASSES = 24
mobilenet.classifier[3] = nn.Linear(num_ftrs, NUM_CLASSES)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mobilenet = mobilenet.to(device)


In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.classifier[3].parameters(), lr=0.001)

EPOCHS = 10

for epoch in range(EPOCHS):
    mobilenet.train()
    total_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = mobilenet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Training loss: {total_loss/len(train_loader)}")

    # Validation accuracy
    mobilenet.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = mobilenet(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Validation accuracy: {100.0 * correct / total}%")


In [ ]:
torch.save(mobilenet.state_dict(), 'asl_recognition_model.pth')
